In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# If the icetemperature library is not in your PYTHONPATH, you will not be able to load those functions
# Check and update here if necessary
import sys
cs_dir = '../'
if cs_dir not in sys.path:
    sys.path.append(cs_dir)

# Import the ice temperature model and relevant constants
from icetemperature.lib.model import ice_temperature
from icetemperature.lib.analytical_functions import conductivity, heat_capacity
from icetemperature.lib.constants import constants
const = constants()

# Interpolator for density profile
from scipy.interpolate import interp1d

In [ ]:
### Load Relevant Datasets

# Measured temperature profile (IceCube/AMANDA)
z_data,T_data,C_data = np.transpose(np.load('../data/compiled_insitu_icetemp.npy'))

# Scaled paleoclimate input from spicecore
model_accumulation = np.load('../data/paleoAccumulation.npy')
model_airTemp = np.load('../data/paleoAirTemperature.npy')
ts = model_accumulation[0]
adot = model_accumulation[1]
Tsurf = model_airTemp[1]

# Density data from spice core
data = np.genfromtxt('../data/SP19_Depth_Age.csv',skip_header=1,delimiter=',')
z_rho = data[:,0]
rho_data = data[:,5]

# Ice thickness from radar through IceCube array
H = 2880.
z_data[-1] = -H

# Assume that the bed is thawed
T_data[-1] = const.rho*const.g*const.beta*H

# Plot the figure to double check that the input data look good
plt.figure(figsize=(6,6))
ax1 = plt.subplot(211)
plt.tick_params(labelbottom=False)
ax1.plot((ts-np.max(ts))/1000.,Tsurf,'k')
plt.ylabel('Temperature ($^\circ$C)')
ax2 = plt.subplot(212)
ax2.plot((ts-np.max(ts))/1000.,adot,'k')
plt.ylabel('Accumulation (m/yr)')
plt.xlabel('kyr');

In [ ]:
# Instantiate the model class
m = ice_temperature()

# Set the time step to 5 years and subsample the paleoclimate data to match
m.ts = ts[::5]*const.spy
m.adot = adot[::5]/const.spy
m.Ts = Tsurf[::5]

# Set the model parameters
m.H = H
m.qgeo = .06
m.p = 1000.
m.Udef = 0.0/const.spy
m.Uslide = 9.3/const.spy

# Longitudinal advection forcing
m.dTs = 1.5e-5
m.dH = 0.003
m.da = 0.
m.flags.append('weertman_vel')

# Thermal conductivity should be temperature and density dependent; set here
m.flags.append('temp-dependent')
m.initial_conditions()
rho_interpolator = interp1d(m.H-z_rho,rho_data,fill_value='extrapolate')
m.rho = 1000.*rho_interpolator(m.z)
m.k = conductivity(m.T.copy(),m.rho)
m.Cp = heat_capacity(m.T.copy())

# Initialize the model to steady state
m.source_terms()
m.stencil()
m.run_to_steady_state()

# Run the model
m.flags.append('save_all')
m.run()

In [ ]:
# Plot the model output against the measured temperature profile
plt.figure(figsize=(4,4))
plt.plot(T_data,z_data,'k.',label='Measurements')
plt.plot(m.T,m.z-m.H,c='indianred',label='Model')
plt.ylabel('Depth (m)')
plt.xlabel('Temperature ($^\circ$C)')
plt.legend()
plt.tight_layout()